In [1]:
import numpy as np
import pandas as pd
import seaborn as sbn
import matplotlib.pyplot as plt
import warnings

warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv('train.csv')
_df = pd.read_csv('test.csv')

In [3]:
_df['Surname'] = _df['Name'].apply(lambda x: x.split(',')[0])
df['Surname'] = df['Name'].apply(lambda x: x.split(',')[0])
df = pd.merge(df,_df['Surname'])

In [5]:
_df['Title'] = _df['Name'].apply(lambda x: x.split(',')[1].split('.')[0].replace(' ',''))
df['Title'] = df['Name'].apply(lambda x: x.split(',')[1].split('.')[0].replace(' ',''))

# _df['Title'] = _df['Title'].replace('Col','Mr')
# _df['Title'] = _df['Title'].replace('Rev','Mr')
# _df['Title'] = _df['Title'].replace('Dr','Mr')
# _df['Title'] = _df['Title'].replace('Dona','Mrs')
# _df['Title'] = _df['Title'].replace('Miss','Mrs')
# _df['Title'] = _df['Title'].replace('Ms','Mrs')

# df['Title'] = df['Title'].replace('Dr','Mr')
# df['Title'] = df['Title'].replace('Capt','Mr')
# df['Title'] = df['Title'].replace('Dr','Mr')
# df['Title'] = df['Title'].replace('Miss','Mrs')
# df['Title'] = df['Title'].replace('Miss','Mrs')

# df['Title'] = df['Title'].apply(lambda x: 'erkek' if x == 'Mr' else ('kadın' if x == 'Mrs' else 'Others'))

In [7]:
_list = ['Master','Mr','Mrs']
df_pivot = df.pivot_table('Survived', index='Surname',columns='Title')
for i in _list: 
    df_pivot[i] = df.pivot_table('Survived', index='Surname',columns='Title')[i].apply(lambda x: 0 if x != 1 else 1)

df_pivot = df_pivot.reset_index()
_df = pd.merge(_df, df_pivot, on='Surname',how='outer')
_df = _df.sort_values('PassengerId')

In [ ]:
df['Embarked'] = df['Embarked'].fillna('S')
_df['Embarked'] = _df['Embarked'].fillna('S')

Embarked Rate

In [ ]:
_list = ['female','male']
__list = ['C','Q','S']
for i in _list:
    for j in range(3):
        index = df[df['Sex'] == i][df['Embarked'] == __list[j]].index
        _index = _df[_df['Sex'] == i][_df['Embarked'] == __list[j]].index
        value = df.pivot_table('Survived',index='Embarked',columns='Sex')[i][j]
        df.loc[index,'Embarked_rate'] = round(value,1)
        _df.loc[_index,'Embarked_rate'] = round(value,1)

Pclass Rate

In [ ]:
df.pivot_table('Survived',index='Pclass',columns='Sex')

_list = ['female','male']
for i in _list:
    for j in range(3):
        index = df[df['Sex'] == i][df['Pclass'] == (j+1)].index
        _index = _df[_df['Sex'] == i][_df['Pclass'] == (j+1)].index
        value = df.pivot_table('Survived',index='Pclass',columns='Sex')[i][j+1]
        df.loc[index,'Pclass_rate'] = round(value,1)
        _df.loc[_index,'Pclass_rate'] = round(value,1)

Rate for Sex

In [ ]:
# women_rate = df.groupby('Sex')[['Survived']].mean().iloc[0].values[0]
# men_rate = df.groupby('Sex')[['Survived']].mean().iloc[1].values[0]
# df['Rate'] = df['Sex'].apply(lambda x: men_rate if x == 'male' else women_rate)
# _df['Rate'] = df['Sex'].apply(lambda x: men_rate if x == 'male' else women_rate)

Fill nan Age values

In [ ]:
def fillAge(df): ## title
    start = df.groupby(['Sex','Pclass'])[['Age']].agg(lambda x: x.mean() + x.std()) ##.unstack()
    end = df.groupby(['Sex','Pclass'])[['Age']].agg(lambda x: x.mean() - x.std()) ##.unstack()

    _list = ['female','male']
    for i in _list:
        for j in range(3):
            x = start.loc[i].iloc[j].values[0]
            y = end.loc[i].iloc[j].values[0]
            length = len(df[df['Sex'] == i][df['Pclass'] == (j+1)][np.isnan(df['Age'])]) 
            index = df[df['Sex'] == i][df['Pclass'] == (j+1)][df['Age'].isna()].index
            random_values = np.random.randint(y,x,length)
            df['Age'].loc[index] = random_values

    df['Age'] = df['Age'].astype(int)

fillAge(df)
fillAge(_df)    

Categorical Age

In [ ]:
df.loc[ df['Age'] <= 16, 'Age'] = 0
df.loc[(df['Age'] > 16) & (df['Age'] <= 32), 'Age'] = 1
df.loc[(df['Age'] > 32) & (df['Age'] <= 48), 'Age'] = 2
df.loc[(df['Age'] > 48) & (df['Age'] <= 64), 'Age'] = 3
df.loc[ df['Age'] > 64, 'Age'] = 4

_df.loc[ _df['Age'] <= 16, 'Age'] = 0
_df.loc[(_df['Age'] > 16) & (_df['Age'] <= 32), 'Age'] = 1
_df.loc[(_df['Age'] > 32) & (_df['Age'] <= 48), 'Age'] = 2
_df.loc[(_df['Age'] > 48) & (_df['Age'] <= 64), 'Age'] = 3
_df.loc[ _df['Age'] > 64, 'Age'] = 4

fig = plt.figure(figsize=(12,5))
sbn.barplot(x='Age', y='Survived', data=df)
plt.show()

Categorical Fare

In [ ]:
df['CatFare'] = pd.qcut(df['Fare'],4)
_df['CatFare'] = pd.qcut(_df['Fare'],4)

fig = plt.figure(figsize=(12,5))
sbn.barplot(x='CatFare', y='Survived', data=df)
plt.show()

In [ ]:
from sklearn.preprocessing import LabelEncoder

label = LabelEncoder()
df['CatFare'] = label.fit_transform(df['CatFare'])
_df['CatFare'] = label.fit_transform(_df['CatFare'])

Categorical Sex and Embarked

In [ ]:
df['Sex'] = df['Sex'].map({'female':1, 'male':0}).astype('int')
_df['Sex'] = _df['Sex'].map({'female':1, 'male':0}).astype('int')

df['Embarked'] = df['Embarked'].map({'S':0, 'C':1, 'Q':2}).astype('int')
_df['Embarked'] = _df['Embarked'].map({'S':0, 'C':1, 'Q':2}).astype('int')

In [ ]:
fig = plt.figure(figsize=(12,5))
sbn.barplot(x='Sex', y='Survived', data=df)
plt.show()

In [ ]:
fig = plt.figure(figsize=(12,5))
sbn.barplot(x='Embarked', y='Survived', data=df)
plt.show()

is Alone?

In [ ]:
df['Alone'] = df['SibSp'] + df['Parch']
df['Alone'] = df['Alone'].apply(lambda x: 1 if x != 0 else 0)

_df['Alone'] = _df['SibSp'] + _df['Parch']
_df['Alone'] = _df['Alone'].apply(lambda x: 1 if x != 0 else 0)

In [ ]:
fig = plt.figure(figsize=(12,5))
sbn.barplot(x='Alone', y='Survived', data=df)
plt.show()

Family Size

In [ ]:
df['FamilySize'] = df['SibSp'] + df['Parch'] ## 2 gruba böl
_df['FamilySize'] = _df['SibSp'] + _df['Parch']

In [ ]:
fig = plt.figure(figsize=(12,5))
sbn.barplot(x='FamilySize', y='Survived', data=df)
plt.show()

is Cabin?

In [ ]:
df['isCabin'] = df["Cabin"].apply(lambda x: 0 if type(x) == float else 1)
_df['isCabin'] = _df["Cabin"].apply(lambda x: 0 if type(x) == float else 1)

In [ ]:
fig = plt.figure(figsize=(12,5))
sbn.barplot(x='isCabin', y='Survived', data=df)
plt.show()

Title

In [ ]:
df['Title'] = df['Name'].apply(lambda x: x.split(',')[1].split('.')[0].replace(' ',''))
_df['Title'] = _df['Name'].apply(lambda x: x.split(',')[1].split('.')[0].replace(' ',''))

In [ ]:
_list = ['Mr', 'Mrs', 'Master']
df['Title'].replace(['Lady', 'Miss', 'Ms', 'Mlle', 'Mme'], 'Mrs', inplace=True)
df['Title'].replace(['Sir', 'Rev'], 'Mr', inplace=True)
df['Title'][~df.Title.isin(_list)] = 'Others'

_df['Title'].replace(['Lady', 'Miss', 'Ms', 'Mlle', 'Mme'], 'Mrs', inplace=True)
_df['Title'].replace(['Sir', 'Rev'], 'Mr', inplace=True)
_df['Title'][~_df.Title.isin(_list)] = 'Others'

In [ ]:
fig = plt.figure(figsize=(12,5))
sbn.barplot(x='Title', y='Survived', data=df)
plt.show()

In [ ]:
df.drop(['Name'],axis=1,inplace=True)
_df.drop(['Name'],axis=1,inplace=True)

Categorical Title

In [ ]:
df['Title'] = df['Title'].map({'Mr':0, 'Mrs':1,'Master':2,'Others':3}).astype('int')
_df['Title'] = _df['Title'].map({'Mr':0, 'Mrs':1,'Master':2,'Others':3}).astype('int')

Drop columns

In [ ]:
df.drop(['Parch','SibSp','Cabin','Fare','PassengerId','Ticket'], axis=1,inplace=True)
_df.drop(['Parch','SibSp','Cabin','Fare','PassengerId','Ticket'], axis=1,inplace=True)

In [ ]:
df.to_csv('df_train.csv',index=False)
_df.to_csv('df_test.csv',index=False)